In [10]:
from params import *

In [11]:
from knowknow import *

In [12]:
from knowknow.datasources import wos

In [13]:
prelim_count = Dataset(f"{ident}-wos-all")

loading variable anthro-wos-all/groups from disk


In [14]:
filtered = Dataset(f"{ident}-wos-journalFilter")
filtered.groupings = [] # clear... not yet

loading variable anthro-wos-journalFilter/groups from disk
loading variable anthro-wos-journalFilter/group_reps from disk


In [15]:
final = Dataset(f'{ident}-wos-clean')

loading variable anthro-wos-clean/groups from disk


# groupings

In [16]:
prelim_count.groupings.append((
    'fj',{
        "yearbook series vol 39: supplement 23;to the american journal of physical anthropology": "yearbook of physical anthropology", 
        "yearbook of physical anthropology: vol 46": "yearbook of physical anthropology", 
        "yearbook of physical anthropology": "yearbook of physical anthropology", 
        "yearbook of physical anthropology 1999, vol 42": "yearbook of physical anthropology", 
        "yearbook of physical anthropology, 1995": "yearbook of physical anthropology", 
        "yearbook of physical anthropology, vol 36 1993": "yearbook of physical anthropology", 
        "yearbook of physical anthropology, vol 41 - 1998": "yearbook of physical anthropology", 
        "yearbook of physical anthropology, vol 43 2000": "yearbook of physical anthropology", 
        "yearbook of physical anthropology, vol 44": "yearbook of physical anthropology", 
        "yearbook of physical anthropology, vol 45": "yearbook of physical anthropology", 
        "yearbook of physical anthropology, vol 47": "yearbook of physical anthropology", 
        "yearbook of physical anthropology": "yearbook of physical anthropology", 
        "yearbook series vol 39: supplement 23;": "yearbook of physical anthropology", 
        "american journal of physical anthropology-new series": "american journal of physical anthropology",
    }
))

# Counting over the entire dataset

In [7]:
cnt = wos.counter(
    wos_txt_base = base_dir, wos_type = 'txt',
    output_database = prelim_count.name,
    RUN_EVERYTHING = False,
    name_blacklist=name_blacklist
)

cnt.run()

File 0/305: anthro.leq2010.23001-23500.txt
Document: 0
Citations: 0
Journals: []
File 50/305: anthro.leq2010.88501-89000.txt
Document: 24780
Citations: 531675
Journals: [('journal of archaeological science', 1331), ('man', 937), ('american anthropologist', 933), ('anthropos', 692), ('antiquity', 601), ('collegium antropologicum', 514), ('archaeological and anthropological sciences', 505), ('american journal of human biology', 505), ('american antiquity', 474), ('american journal of physical anthropology', 462), ('race & class', 451), ('anthropologie', 417), ('human biology', 416), ('human organization', 392), ('human ecology', 385)]
File 100/305: anthro.geq2011.14501-15000.txt
Document: 49780
Citations: 918202
Journals: [('journal of archaeological science', 2348), ('american journal of physical anthropology', 2203), ('man', 1602), ('american anthropologist', 1543), ('antiquity', 1372), ('american antiquity', 1214), ('anthropos', 1161), ('american journal of human biology', 1063), ('hu

In [8]:
prelim_count.set_attribute('RELIABLE_DATA_STARTS_HERE', 1900)
prelim_count.set_attribute('RELIABLE_DATA_ENDS_HERE', 2015)
prelim_count.set_attribute('type','wos')

In [9]:
max( prelim_count.items('fy') )

loading variable anthro-wos-all/ind ___ fy from disk


2022

# Examine journals to determine what to drop

In [18]:
zero_sofars = []
# how I decide whether to keep a journal
def filt(j):
    
    try:
        tt = prelim_count.trend('fj', j)
    except invalid_entry:
        return False

    # get the years for which there is a positive count of BIBLIOGRAPHIES
    ys = set(y for y,c in tt._c.items() if c > 0)

    # this is a weird error, probably requires me to rerun (rework for efficiency) my counting algos
    # might even be reduced once I actually filter the journals
    if not len(ys):
        #raise Exception("no years for journal %s" % j)
        return False

    # it must have been around for at least 10 years, with their first year being sometime before 1995
    if not(
        (max(ys)-min(ys) >= 10 and min(ys) <= 1995 and max(ys) >= 2015)
    ): return False

    # 5 years consecutively, then drop
    ever_nonzero = False
    zero_sofar = 0
    zero_sofar_M = 0
    for y in range(tt.data_start, tt.data_end):
        c = tt._c[ y ]
        
        if c > 0:
            ever_nonzero = True

        if not ever_nonzero:
            continue

        if c == 0:
            zero_sofar += 1
        else:
            zero_sofar_M = max(zero_sofar_M, zero_sofar)
            zero_sofar = 0

        if zero_sofar >= 5:
            return False
        
    # at least 100 papers published total
    if not(
        prelim_count(fj=j).docs >= 100
    ): return False
    
    zero_sofars.append(zero_sofar_M)
    if zero_sofar_M > 0:
        print(j,zero_sofar_M)

    return True

In [19]:
journals = list(prelim_count.items('fj'))

print( "There are %s journals total in `%s`" % (len(journals), prelim_count.name) )

keep, drop = set(), set()
for j in journals:
    if filt(j):
        keep.add(j)
    else:
        drop.add(j)
        
print( "Only %s meet my standards" % (len(keep) ) )

loading variable anthro-wos-all/ind ___ fj from disk
There are 200 journals total in `anthro-wos-all`
loading variable anthro-wos-all/ind ___ fj.fy from disk
loading variable anthro-wos-all/ind ___ fy from disk
loading variable anthro-wos-all/doc ___ fj from disk
africa 2
anthropos 3
cultural anthropology 2
homo-journal of comparative human biology 2
journal of anthropological research 2
journal of world prehistory 3
oceania 2
Only 43 meet my standards


In [20]:
Counter(zero_sofars)

Counter({2: 5, 0: 36, 3: 2})

## journals to keep

In [21]:
print(", ".join(sorted(keep)))

africa, american anthropologist, american antiquity, american ethnologist, american journal of human biology, american journal of physical anthropology, annals of human biology, anthropological quarterly, anthropological science, anthropology & education quarterly, anthropos, anthrozoos, antiquity, arctic anthropology, bijdragen tot de taal- land- en volkenkunde, comparative studies in society and history, critique of anthropology, cultural anthropology, cultural studies, culture medicine and psychiatry, current anthropology, ethnohistory, ethos, homo-journal of comparative human biology, human biology, human ecology, human nature-an interdisciplinary biosocial perspective, human organization, international journal of osteoarchaeology, journal of anthropological archaeology, journal of anthropological research, journal of archaeological science, journal of family history, journal of historical sociology, journal of human evolution, journal of peasant studies, journal of the royal anthr

## journals to drop

In [22]:
print(",".join(sorted(drop)))

acta ethnographica,acta ethnographica academiae scientiarum hungaricae,african archaeological review,aibr-revista de antropologia iberoamericana,annals of anthropological practice,annual review of anthropology,anthropologiai kozlemenyek-anthropological communications,anthropologica,anthropologica-canada,anthropological forum,anthropological journal of canada,anthropological journal of european cultures,anthropological linguistics,anthropological notebooks,anthropological papers of the american museum of natural history,anthropological papers of the museum of anthropology of the university;michigan,anthropological theory,anthropologie,anthropologie et sante-revue internationale francophone d anthropologie;de la sante,anthropologie-international journal of human diversity and evolution,anthropologischer anzeiger,anthropologist,anthropology & medicine,anthropology and archeology of eurasia,anthropology in action-journal for applied anthropology in policy and;practice,anthropology of consc

# compute counts over just these selected journals

In [15]:
cnt = wos.counter(
    wos_txt_base = base_dir, wos_type = 'txt',
    output_database = filtered.name,
    RUN_EVERYTHING = False,
    name_blacklist=name_blacklist,
    journals_filter=keep
)

cnt.run()

File 0/305: anthro.leq2010.23001-23500.txt
Document: 0
Citations: 0
Journals: []
File 50/305: anthro.leq2010.88501-89000.txt
Document: 24780
Citations: 314643
Journals: [('journal of archaeological science', 1331), ('man', 937), ('american anthropologist', 933), ('anthropos', 692), ('antiquity', 601), ('collegium antropologicum', 514), ('archaeological and anthropological sciences', 505), ('american journal of human biology', 505), ('american antiquity', 474), ('american journal of physical anthropology', 462), ('race & class', 451), ('anthropologie', 417), ('human biology', 416), ('human organization', 392), ('human ecology', 385)]
File 100/305: anthro.geq2011.14501-15000.txt
Document: 49780
Citations: 558763
Journals: [('journal of archaeological science', 2348), ('american journal of physical anthropology', 2203), ('man', 1602), ('american anthropologist', 1543), ('antiquity', 1372), ('american antiquity', 1214), ('anthropos', 1161), ('american journal of human biology', 1063), ('hu

In [16]:
filtered.set_attribute('RELIABLE_DATA_STARTS_HERE', 1900)
filtered.set_attribute('RELIABLE_DATA_ENDS_HERE', 2015)
filtered.set_attribute('type','wos')

# now compute summary statistics about citations.

There are many we will want to exclude for almost all analyses, in order to keep more cooccurrence counts

In [17]:
if True:
    t = ttDF(
        dataset_name = filtered.name,
        dtype = 'c',
        debug=False
    ).run()

loading variable anthro-wos-journalFilter/groups from disk
loading variable anthro-wos-journalFilter/ind ___ c from disk
Item 0/1383166 (#Blacklivesmatter, O.|2016|11 maj misc black li)
Current memory usage is 247.695761MB; Peak was 649.763443MB
defaultdict(<class 'int'>, {})
loading variable anthro-wos-journalFilter/doc ___ c from disk
loading variable anthro-wos-journalFilter/ind ___ c.fy from disk
loading variable anthro-wos-journalFilter/ind ___ fy from disk
Item 50000/1383166 (Asch, D.|1978|anthr papers museum,v67,p301)
Current memory usage is 994.943354MB; Peak was 1176.471147MB
defaultdict(<class 'int'>, {'at least one citation': 50000, 'less than 5 citations... dropped.2': 47302, 'not enough parts': 26, 'passed tests pre-blacklist': 1857, 'less than 5 citations... dropped.': 681, 'invalid entry': 132, 'first citation before pub date': 2})
Item 100000/1383166 (Bender, D.|1970|man,v5,p71)
Current memory usage is 1110.486847MB; Peak was 1186.5985MB
defaultdict(<class 'int'>, {'at 

In [18]:
len( filtered.load_variable('c.ysum') )

loading variable anthro-wos-journalFilter/c.ysum from disk


58727

# and group those which make the cut

In [7]:
# note that unfortunately this currently requires YSUM
#   and thus will be based on the filters embedded in that...
#   AND these decisions will propagate further, because counters only consider if they're in the groups
# some decisions need to be made here...
from grouper import Grouper
g = Grouper(dataset=filtered)
g.run()

loading variable anthro-wos-journalFilter/c.ysum from disk
58727 strings total...
sample articles: ['2-2,, D.|1997|', '2-T,, D.|2000|', 'Aaby, B.|1994|veg hist archaeobot,v3,p167', 'Abate, N.|1996|diabetes,v45,p1684', 'Abbate, E.|1998|nature,v393,p458', 'Abbate, E.|2012|quatern int,v267,p3', 'Abbes, F.|2001|syria,v78,p5', 'Abbes, F.|2003|j non-cryst solids,v323,p162', 'Abbie, A.|1947|j anat,v81,p233', 'Abbie, A.|1951|oceania,v22,p91']
sample books: ['Aaris-Sorensen, K.|danmarks forhistoris', 'Aaris-Sorensen, K.|foss strata', 'Aarne, A.|types folktale', 'Abadie, R.|professional guinea', 'Abbas, A.|hong kong culture po', 'Abbie, A.|original austr', 'Abbott, A.|system professions e', 'Abbott, D.|centuries decline ho', 'Abbott, D.|ceramic production a', 'Abbott, D.|ceramics community o']
35972 articles, 22755 books to group
3205 books have some connection to others in a group
21010 groups total
[(11614, 8), (11683, 8), (8746, 7), (18581, 7), (20717, 7), (3519, 6), (6303, 6), (16905, 6), (1

# recount on this focused group, grouping citations

In [23]:
#toload = ['fj','fy','ty','c.fy','c.fj','fj.fy','c','fy.ty','fj.ty','ta','fy.ta','fj.ta','c.fj.fy','ffa','ffa.fy','ffa.fj','c.ffa','fa','fa.fy','fa.fj','c.fa']

cnt = wos.counter(
    wos_txt_base = base_dir, wos_type = 'txt',
    output_database = final.name,
    RUN_EVERYTHING = True,
    name_blacklist=name_blacklist,
    journals_filter= keep,
    groups= filtered.load_variable('groups'),
    group_reps= filtered.load_variable('group_reps'),
    #debug=True
)

cnt.run()
"""
to just compute ages, pulling from another set of counts, for debugging //
for k in toload:
    cnt.doc[k] = final.by(*k.split(".")).docs
    cnt.ind[k] = final.by(*k.split(".")).cits

cnt.count_ages()
"""

final.set_attribute('RELIABLE_DATA_STARTS_HERE', 1900)
final.set_attribute('RELIABLE_DATA_ENDS_HERE', 2015)
final.set_attribute('type','wos')

loading variable anthro-wos-journalFilter/groups from disk
loading variable anthro-wos-journalFilter/group_reps from disk
File 0/305: anthro.leq2010.23001-23500.txt
Document: 0
Citations: 0
Journals: []
File 50/305: anthro.leq2010.88501-89000.txt
Document: 24780
Citations: 42824
Journals: [('journal of archaeological science', 1331), ('man', 937), ('american anthropologist', 933), ('anthropos', 692), ('antiquity', 601), ('collegium antropologicum', 514), ('archaeological and anthropological sciences', 505), ('american journal of human biology', 505), ('american antiquity', 474), ('american journal of physical anthropology', 462), ('race & class', 451), ('anthropologie', 417), ('human biology', 416), ('human organization', 392), ('human ecology', 385)]
File 100/305: anthro.geq2011.14501-15000.txt
Document: 49780
Citations: 53528
Journals: [('journal of archaeological science', 2348), ('american journal of physical anthropology', 2203), ('man', 1602), ('american anthropologist', 1543), (

# do the simple string grouper, finally...

In [24]:
from grouper import SimpleStringGrouper
SimpleStringGrouper(final).run()

loading variable anthro-wos-clean/ind ___ c from disk
